## Exercise 4

(and parts of exercise 5)

In [2]:
import pandas as pd
import numpy as np
from astropy.units import Mpc, km, s, erg, K, L_sun, cm, g, kpc
from astropy.constants import G, m_p, k_B

# Very quick and dirty script, please don't try at home!
# Would be better to work with astropy Tables to keep track of the units. Sorrey!

data = {
    "Cluster": ["Virgo", "Coma"],
    "distance": [17, 90],
    "angle": [10, 4],
    "L": [1.2e12, 5e12],
    "sigma": [670, 980],
    "Lx": [1.5e43, 5e44],
    "Tx": [7e7, 8.8e7]
}

def calc_R(objs) -> float:
    dist, angle = objs
    angle = angle/2
    return dist*np.tan(np.deg2rad(angle))

def calc_Mvir(objs) -> float:
    """Calculate the virial mass using the given formula, in solar masses."""
    sig, r = objs
    sig *= km/s
    r *= Mpc
    return (sig**2*5*r/G).to_value("Msun")

def calc_ne(objs):
    Lx, Tx, r = objs
    Lx *= erg/s
    Tx *= K
    r *= Mpc
    alph = 5.95e-27 * cm**5*g/s**3/K**0.5
    return np.sqrt(Lx/(alph*r**3*np.sqrt(Tx))).to_value("cm**(-3)")

def calc_Mx(objs):
    n_e, r = objs
    n_e *= cm**(-3)
    r *= Mpc
    return (4/3*np.pi*r**3*n_e*m_p).to_value("Msun")

def calc_Etherm(objs):
    n_e, Tx, r = objs
    n_e *= cm**(-3)
    Tx *= K
    r *= Mpc
    return (3/2*k_B*Tx*n_e*4/3*np.pi*r**3).to_value("erg")

def calc_tau(objs):
    Etherm, Lx = objs
    Lx *= erg/s
    Etherm *= erg
    return (Etherm/Lx).to_value("yr")

def calc_crossing_time(objs):
    sig, r = objs
    sig *= km/s
    r *= Mpc
    return (r/sig).to_value("yr")

mass_to_light_ratio = 3

age_of_universe= 13e9  # around 13 Gyr

df = pd.DataFrame(data)
df["R"] = df[["distance", "angle"]].apply(calc_R, axis=1)  # The radius of the cluster
df["Mvir"] = df[["sigma", "R"]].apply(calc_Mvir, axis=1)  # The virial mass of the cluster
df["Mlum"] = df["L"] * mass_to_light_ratio  # The luminous stellar mass of the cluster
df["Mvir/Mlum"] = df["Mvir"] / df["Mlum"]  # the ratio between virial and luminous mass
df["ne"] = df[["Lx", "Tx", "R"]].apply(calc_ne, axis=1)  # The electron density
df["Mx"] = df[["ne", "R"]].apply(calc_Mx, axis=1)  # The X-ray mass
df["Mbary/Mvir"] = ( df["Mlum"] + df["Mx"]) /  df["Mvir"]  # The DM ratio
df["Etherm"] = df[["ne", "Tx", "R"]].apply(calc_Etherm, axis=1)  # The thermal energy in ergs
df["Lifetime"] = df[["Etherm", "Lx"]].apply(calc_tau, axis=1)  # The lifetime in yr
df["Cross_time"] = df[["sigma", "R"]].apply(calc_crossing_time, axis=1)

print(df.T.to_latex(float_format="${:.3e}$".format, escape=False))
print(df["Lifetime"]/age_of_universe)
print(df["Cross_time"]/age_of_universe)

\begin{tabular}{lll}
\toprule
{} &           0 &           1 \\
\midrule
Cluster    &       Virgo &        Coma \\
distance   &          17 &          90 \\
angle      &          10 &           4 \\
L          & $1.200e+12$ & $5.000e+12$ \\
sigma      &         670 &         980 \\
Lx         & $1.500e+43$ & $5.000e+44$ \\
Tx         & $7.000e+07$ & $8.800e+07$ \\
R          & $1.487e+00$ & $3.143e+00$ \\
Mvir       & $7.762e+14$ & $3.509e+15$ \\
Mlum       & $3.600e+12$ & $1.500e+13$ \\
Mvir/Mlum  & $2.156e+02$ & $2.339e+02$ \\
ne         & $5.583e-05$ & $9.910e-05$ \\
Mx         & $1.902e+13$ & $3.185e+14$ \\
Mbary/Mvir & $2.914e-02$ & $9.504e-02$ \\
Etherm     & $3.277e+62$ & $6.900e+63$ \\
Lifetime   & $6.923e+11$ & $4.373e+11$ \\
Cross_time & $2.171e+09$ & $3.136e+09$ \\
\bottomrule
\end{tabular}

0    53.255213
1    33.639508
Name: Lifetime, dtype: float64
0    0.166966
1    0.241215
Name: Cross_time, dtype: float64


In [8]:
radii_and_vel_disps = [(1.5*Mpc*2, 670 *km/s),  # Virgo
(3.1*Mpc*2, 980 * km/s), # Coma
(100 * Mpc, 1000 * km/s), # Superclusters
(10 * kpc, 250 * km/s), # Ellipticals
(100 * kpc, 250 *km/s)
]
for diameter, vel in radii_and_vel_disps:
    time = (diameter /vel).to("yr")
    print(f"{diameter = :.1f}, {vel = :.0f} --> {time = :.2e}")

diameter = 3.0 Mpc, vel = 670 km / s --> time = 4.38e+09 yr
diameter = 6.2 Mpc, vel = 980 km / s --> time = 6.19e+09 yr
diameter = 100.0 Mpc, vel = 1000 km / s --> time = 9.78e+10 yr
diameter = 10.0 kpc, vel = 250 km / s --> time = 3.91e+07 yr
diameter = 100.0 kpc, vel = 250 km / s --> time = 3.91e+08 yr
